In [2]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from pymongo import MongoClient
import datetime
import random
import re
import json

In [3]:
client = MongoClient('163.14.137.58', 8099)
client_db = client['News']
client_col = client_db['Wine_udn酒版']

### 從資料庫抓出最後時間

In [4]:
mydoc = client_col.find().sort("time", -1).limit(1)
for i in mydoc:
    end_time = i['time']
end_time

'2021-02-18 12:13:06'

In [2]:
base_url = "https://udn.com/api/more"    
channelId = 2
type_ = 'searchword'
search_ = '%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2'
query = f"&id=search:{search_}&channelId={channelId}&type={type_}"
news_list_url = base_url + '?' + query
print(news_list_url)

https://udn.com/api/more?&id=search:%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2&channelId=2&type=searchword


In [8]:
def get_news_list(end_time):
    ### 參考資料：https://blog.jiatool.com/posts/udn_spider/
    
    """
    爬取新聞列表
    page_num:網頁頁數
    end_time:DB最後時間
    """
    base_url = "https://udn.com/api/more"

    news_list = []
    
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
    }
    page_num = 50
    for page in tqdm(range(page_num)):
        ### 找出 Ajax 去請求的網址，然後照他的請求方式、參數來取得資料。
        
        channelId = 2
        type_ = 'searchword'
        search_ = '%E2%80%BB%20%E6%8F%90%E9%86%92%E6%82%A8%EF%BC%9A%E7%A6%81%E6%AD%A2'
        query = f"page={page+1}&id=search:{search_}&channelId={channelId}&type={type_}"
        news_list_url = base_url + '?' + query
#         print(news_list_url)###
        
        r = requests.get(news_list_url, headers=HEADERS)
        news_data = r.json()
        news_list.extend(news_data['lists'])
        time.sleep(random.uniform(1, 2))
    
    news_udn = [] ###網址是udn.com，主要抓這邊的資料

    
    for i in news_list:
        if i['time']['date'] > end_time:
            ####
            json_ = {
                    'title':i['title'].replace('\u2003',''),
                    'link':i['titleLink'],
                    'cateLink':i['cateLink'],
                    'cateTitle':i['cateTitle'],
                    'time':i['time']['date'],
                }
#             if i['titleLink'].split('/')[2] == 'udn.com':
            news_udn.append(json_)
    return news_udn


In [25]:
def get_news_home_content(url):

    """
    爬取網頁內文
    url:網址
    """
    
    HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36',
    }
    r = requests.get(url, headers=HEADERS)
    sp=BeautifulSoup(r.text,'lxml')

    try:
        author=sp.find('span','article-content__author').text
        Author = (re.sub(r'[\n ]*', '', author))
    except AttributeError:
        Author = 'None'
    
    try:
        ad = ".innity-apps-underlay-ad {z-index: 34 !important; }.innity-apps-underlay-ad ~ .header {z-index: 35;}.innity-apps-underlay-ad ~ .main-content .inline-ads { background: transparent;}.article-content__editor video {max-width: 100%;} /* to be remove*/    googletag.cmd.push(function() { googletag.display('ads-inline'); });"
        content = sp.find('section','article-content__editor').text
        Content = (re.sub(r'[\n\r]*', '', content).replace(ad, ''))
    except AttributeError:
        Content = 'None'
    json_ = {
        
        'author' : Author,
        'content' : Content
        
    }
    return json_

In [9]:
test_list1 = get_news_list(end_time)

100%|██████████| 50/50 [01:55<00:00,  2.30s/it]


[{'title': '捷絲旅台北西門館「每人900」入住！加碼「艋舺雞排」與調酒',
  'link': 'https://udn.com/news/story/7934/5273252',
  'cateLink': 'https://udn.com/news/cate/1013',
  'cateTitle': '旅遊',
  'time': '2021-02-24 13:28:09'},
 {'title': '最高樓韓式烤肉！虎三同進駐信義A13\u3000喝得到Bar Mood 特調',
  'link': 'https://udn.com/news/story/7193/5273071',
  'cateLink': 'https://udn.com/news/cate/1013',
  'cateTitle': '旅遊',
  'time': '2021-02-24 12:44:09'},
 {'title': '最高樓韓式烤肉！虎三同進駐A13 喝得到Bar Mood 特調',
  'link': 'https://udn.com/news/story/7270/5273071',
  'cateLink': 'https://udn.com/news/cate/6649',
  'cateTitle': '生活',
  'time': '2021-02-24 12:44:09'},
 {'title': '陳穎Ying C.／甜點與酒的甜蜜邂逅',
  'link': 'https://udn.com/news/story/7341/5271734',
  'cateLink': 'https://udn.com/news/cate/6643',
  'cateTitle': '評論',
  'time': '2021-02-24 00:50:13'},
 {'title': '到底有多渴？這個網站讓你隨時來點微醺白噪音',
  'link': 'https://udn.com/news/story/7270/5272280',
  'cateLink': 'https://udn.com/news/cate/6649',
  'cateTitle': '生活',
  'time': '2021-02-24 00:05:10'},
 {'titl

In [26]:
news_alcohol = []
news_accident = []

for i in tqdm(range(len(test_list1))):
    if '酒駕' in test_list1[i]['title']:
        news_accident.append(test_list1[i])
    elif '酒測' in test_list1[i]['title']:
        news_accident.append(test_list1[i])
    elif '酒瘋' in test_list1[i]['title']:
        news_accident.append(test_list1[i]) 
    else:
        news_alcohol.append(test_list1[i])

100%|██████████| 24/24 [00:00<00:00, 7355.20it/s]


In [27]:
news_alcohol

[{'title': '年輕警察不認識老議員 蔡媽福關心選民卻拉扯警遭判刑：絕對要上訴',
  'link': 'https://udn.com/news/story/7321/5257878',
  'cateLink': 'https://udn.com/news/cate/6639',
  'cateTitle': '社會',
  'time': '2021-02-18 12:13:06'},
 {'title': '影／醉男強開賓士與友人拉扯 警拉出壓地制伏公共危險罪送辦',
  'link': 'https://udn.com/news/story/7321/5257629',
  'cateLink': 'https://udn.com/news/cate/6639',
  'cateTitle': '社會',
  'time': '2021-02-18 10:59:06'},
 {'title': '星巴克新口味「鹹焦糖綿雲瑪奇朵」、絕美櫻花杯款上市！加碼19天數位體驗優惠',
  'link': 'https://udn.com/news/story/7185/5255950',
  'cateLink': 'https://udn.com/news/cate/1013',
  'cateTitle': '旅遊',
  'time': '2021-02-18 00:00:40'},
 {'title': '星巴克新口味「鹹焦糖綿雲瑪奇朵」、絕美櫻花杯款上市！加碼19天數位體驗優惠',
  'link': 'https://style.udn.com/style/story/11350/5255950',
  'cateLink': 'https://style.udn.com',
  'cateTitle': 'udnSTYLE',
  'time': '2021-02-18 00:00:40'},
 {'title': '被爆過年上夜店、找美女陪酒\u3000黃曉明回應了',
  'link': 'https://stars.udn.com/star/story/10088/5256213',
  'cateLink': 'https://stars.udn.com',
  'cateTitle': '噓！星聞',
  'time': '2021-

In [30]:
author_content = []
for i in tqdm(range(len(news_alcohol))):
    data = get_news_home_content(news_alcohol[i]['link'])
    author_content.append(data)

100%|██████████| 24/24 [00:40<00:00,  1.67s/it]


In [31]:
for i in tqdm(range(len(author_content))):
    news_alcohol[i]['author'] = author_content[i]['author']
    news_alcohol[i]['content'] = author_content[i]['content']

100%|██████████| 24/24 [00:00<00:00, 49587.83it/s]


In [34]:
with open('%sudn_alcohol.json'%datetime.date.today(),'a') as outfile:
    json.dump(news_alcohol, outfile, ensure_ascii=False)
    outfile.write('\n')

In [35]:
with open('%sudn_alcohol.json'%datetime.date.today()) as f:
    file_data = json.load(f)

client_col.insert_many(file_data)